# Covertype

In [ ]:
'''
Created on 29 мар. 2021 г.

@author: keen
'''
import numpy
import CO2_tree as co2t
import CO2_forest as co2f
import pickle
from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Normalizer
import time
import numpy
from sklearn.datasets import fetch_covtype
from sklearn.pipeline import Pipeline, make_pipeline


datasets = {#"SatImage",
 #"Pendigits",
# "Letter",
# "Protein",
 #"Connect4"",
 #"SensIT",
 "Covertype" 
}

tree_deth = [5]
sratios = [0.]
sratios2 = [1.0]
fratios = [0.8]

C = [1000]
criteria = ["gaussian"]


best_params = {}
for ds in datasets:
    #X = numpy.load("UCI/" + ds + "/X_poly.npy",allow_pickle=True).astype('float32')
    #from sklearn.datasets import fetch_covtype
    X, y = fetch_covtype(return_X_y=True)

    mm = make_pipeline(MinMaxScaler(), Normalizer())
    X = mm.fit_transform(X)
    #idxs = numpy.random.RandomState(seed=42).permutation(range(X.shape[0]))[:50000]

    #X = X[idxs]
    #y = y[idxs] #y = numpy.load("UCI/" + ds + "/y.npy",allow_pickle=True).astype('int64')
    
    #C=[5000,10000]

    best_v_acc = 0.
    for d in tree_deth:
        for sratio in sratios:
            for sratio2 in sratios2:
                for fratio in fratios:
                    for c in C:
                        for ns in [0.]:
                            for cr in criteria:
                                v2_sc = []
                                t_scf = []
                                print ("Dataset (polynomial) ", ds)
                                print ("Test carbon forest with tree deth= ", d+1, " C= ", c, " s ratio ", sratio," s ration 2" ,sratio2," f ratio ",fratio, "criterion=",cr)
                                for rn in range(1):
                                    kf = KFold(n_splits=5, shuffle=True)
                                    itr = 0
                                    for train, test in kf.split(X):
                                        print ("Step", rn)
                                        itr += 1
                                        trc = co2f.CO2_forest(C=c, dual=False,tol = 0.0001,max_iter=1000,kernel=cr,max_deth=d,n_jobs=2,sample_ratio=1.0, feature_ratio = fratio,n_estimators=30,gamma=500.,dropout_low=sratio,dropout_high=sratio2,noise=ns,cov_dr=0.,criteria='gain')
                                        trc.sequential_fit(sparse.csr_matrix(X[train]), y[train])
                                        Y_v_f = trc.sequential_predict(sparse.csr_matrix(X[test]),use_weight = False)
                                        Y_t_f =  trc.sequential_predict(sparse.csr_matrix(X[train]),use_weight = False)
                                        del trc
                                        #print(Y_v)
                
                                        try:
                                            t_scf.append(accuracy_score(y[train],Y_t_f))
                                            v2_sc.append(accuracy_score(y[test],Y_v_f))

                                        except Exception as e:
                                            print("Error:", e)
                                        break    
                                t_scf = numpy.asarray(t_scf)
                                v2_sc = numpy.asarray(v2_sc)
                                
                                vres = v2_sc.mean()
                                
                                if vres > best_v_acc:
                                    best_v_acc = vres
                                    best_params[ds] = [d,fratio,c,cr,sratio,best_v_acc]     
                                        
                                print (cr,t_scf.mean(),vres)    
                                
                                
print ("Best results:", best_params)    

In [ ]:
#C = 2000, gamma = 500
0.973645088627802 0.944769067924236

In [ ]:
0.9712591623656168 0.9441494625784188

In [ ]:
0.9741162499004967 0.9460685180244917

In [ ]:
0.9729329681654185 0.9459308279476434

# Cifar-10

In [1]:
#load Cifar
import pickle
import numpy

from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.pipeline import Pipeline, make_pipeline

prts = []
for i in range(1,6,1):
    with open("cifar/data_batch_" + str(i),'rb') as f:
        prts.append(pickle.load(f, encoding='bytes'))
X_train = numpy.vstack([p[b'data'] for p in prts])
y_train = numpy.hstack([p[b'labels'] for p in prts]) + 1

with open("cifar/test_batch",'rb') as f:
    prt = pickle.load(f, encoding='bytes')
X_test = prt[b'data']
y_test = numpy.asarray(prt[b'labels']) + 1

X = numpy.vstack([X_train,X_test])
mm = make_pipeline(MinMaxScaler(), Normalizer())
X = mm.fit_transform(X)

X_test = X[50000:]
X_train = X[:50000]

X_train_ = X_train
y_train_ = y_train

In [ ]:
from sklearn.metrics import accuracy_score
from thundersvm import *
from sklearn.model_selection import KFold

from sklearn.svm import LinearSVC

import time
import numpy

for gamma in [0.1,1,10]:
    for c in [2000]:
        start_time = time.time()
        kf = KFold(n_splits=5, shuffle=True)
        itr = 0
        res = []
        res2 = []
        for train, test in kf.split(X_train_):       
            X_train = X_train_[train]
            X_test = X_train_[test]
            y_train = y_train_[train]
            y_test = y_train_[test]        
            cl = SVC(C=c,kernel='rbf',tol=0.0001,max_iter=1000,gamma=gamma,verbose=False)
            #cl = LinearSVC(C=c,tol=0.0001,max_iter=1000)

            cl.fit(X_train,y_train)#,sample_weights=deltas)
            y_pred = cl.predict(X_test)
            y_pred2 = cl.predict(X_train)
            res.append(accuracy_score(y_test,y_pred)) 
            res2.append(accuracy_score(y_train,y_pred2)) 
            print (res,res2)
            
        res = numpy.asarray(res)   
        res2 = numpy.asarray(res2) 
        print(gamma,c,res.mean(),res.std())
        print(gamma,c,res2.mean(),res2.std())
        print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#polynomial С= 1000
[0.0961] [0.100975]
[0.0961, 0.1019] [0.100975, 0.099525]
[0.0961, 0.1019, 0.0996] [0.100975, 0.099525, 0.1001]
[0.0961, 0.1019, 0.0996, 0.1967] [0.100975, 0.099525, 0.1001, 0.21075]
[0.0961, 0.1019, 0.0996, 0.1967, 0.1001] [0.100975, 0.099525, 0.1001, 0.21075, 0.099975]
0.01 1000 0.11887999999999999 0.03895537960282251
0.01 1000 0.122265 0.044244995762232814
--- 311.41494631767273 seconds ---
[0.0939] [0.101525]
[0.0939, 0.0951] [0.101525, 0.101225]
[0.0939, 0.0951, 0.0939] [0.101525, 0.101225, 0.101525]
[0.0939, 0.0951, 0.0939, 0.1154] [0.101525, 0.101225, 0.101525, 0.12225]
[0.0939, 0.0951, 0.0939, 0.1154, 0.3019] [0.101525, 0.101225, 0.101525, 0.12225, 0.3133]
0.1 1000 0.14004 0.08134272186249977
0.1 1000 0.14796499999999999 0.08306009661684725
--- 341.511598110199 seconds ---
[0.4995] [0.659825]
[0.4995, 0.5042] [0.659825, 0.66025]
[0.4995, 0.5042, 0.5039] [0.659825, 0.66025, 0.782125]
[0.4995, 0.5042, 0.5039, 0.4992] [0.659825, 0.66025, 0.782125, 0.658975]
[0.4995, 0.5042, 0.5039, 0.4992, 0.4942] [0.659825, 0.66025, 0.782125, 0.658975, 0.6616]
1.0 1000 0.5002000000000001 0.003665514970641924
1.0 1000 0.6845549999999999 0.048792374916578916
--- 502.22666811943054 seconds ---
[0.4709] [0.73395]
[0.4709, 0.4703] [0.73395, 0.79755]
[0.4709, 0.4703, 0.4759] [0.73395, 0.79755, 0.772225]
[0.4709, 0.4703, 0.4759, 0.4821] [0.73395, 0.79755, 0.772225, 0.7923]
[0.4709, 0.4703, 0.4759, 0.4821, 0.4713] [0.73395, 0.79755, 0.772225, 0.7923, 0.733425]
10.0 1000 0.47409999999999997 0.004466766168045954
10.0 1000 0.76589 0.02761925324841351
--- 905.5735156536102 seconds ---

In [ ]:
#rbf C=1000
[0.407] [0.666]
[0.407, 0.4058] [0.666, 0.43165]
[0.407, 0.4058, 0.4134] [0.666, 0.43165, 0.430875]
[0.407, 0.4058, 0.4134, 0.4165] [0.666, 0.43165, 0.430875, 0.669375]
[0.407, 0.4058, 0.4134, 0.4165, 0.3697] [0.666, 0.43165, 0.430875, 0.669375, 0.44235]
0.1 1000 0.40248 0.016862906036623704
0.1 1000 0.52805 0.1140906558399942
--- 1157.2250702381134 seconds ---
[0.4844] [0.5846]
[0.4844, 0.4768] [0.5846, 0.65475]
[0.4844, 0.4768, 0.4762] [0.5846, 0.65475, 0.59345]
[0.4844, 0.4768, 0.4762, 0.4849] [0.5846, 0.65475, 0.59345, 0.587675]
[0.4844, 0.4768, 0.4762, 0.4849, 0.4887] [0.5846, 0.65475, 0.59345, 0.587675, 0.58505]
1 1000 0.4822 0.004889580759124449
1 1000 0.6011049999999999 0.027007139796727857
--- 359.60684418678284 seconds ---
[0.5287] [0.8176]
[0.5287, 0.5179] [0.8176, 0.7553]
[0.5287, 0.5179, 0.5031] [0.8176, 0.7553, 0.757225]
[0.5287, 0.5179, 0.5031, 0.5139] [0.8176, 0.7553, 0.757225, 0.7565]
[0.5287, 0.5179, 0.5031, 0.5139, 0.5259] [0.8176, 0.7553, 0.757225, 0.7565, 0.774375]
10 1000 0.5179 0.009117894493796245
10 1000 0.7722 0.023758119664653604
--- 442.2167375087738 seconds ---

#rbf С=500
[0.4013] [0.4111]
[0.4013, 0.3418] [0.4111, 0.3704]
[0.4013, 0.3418, 0.3673] [0.4111, 0.3704, 0.515425]
[0.4013, 0.3418, 0.3673, 0.4095] [0.4111, 0.3704, 0.515425, 0.663125]
[0.4013, 0.3418, 0.3673, 0.4095, 0.3756] [0.4111, 0.3704, 0.515425, 0.663125, 0.515125]
0.1 500 0.3791 0.024330145909961156
0.1 500 0.495035 0.10163914182046205
--- 898.1008167266846 seconds ---
[0.4723] [0.541875]
[0.4723, 0.4765] [0.541875, 0.5479]
[0.4723, 0.4765, 0.4706] [0.541875, 0.5479, 0.541875]
[0.4723, 0.4765, 0.4706, 0.4707] [0.541875, 0.5479, 0.541875, 0.566475]
[0.4723, 0.4765, 0.4706, 0.4707, 0.4726] [0.541875, 0.5479, 0.541875, 0.566475, 0.540125]
1 500 0.47253999999999996 0.002139719607799103
1 500 0.54765 0.009775172632746685
--- 335.5505347251892 seconds ---
[0.5206] [0.753825]
[0.5206, 0.521] [0.753825, 0.753825]
[0.5206, 0.521, 0.5133] [0.753825, 0.753825, 0.7546]
[0.5206, 0.521, 0.5133, 0.5313] [0.753825, 0.753825, 0.7546, 0.844275]
[0.5206, 0.521, 0.5133, 0.5313, 0.5162] [0.753825, 0.753825, 0.7546, 0.844275, 0.753025]
10 500 0.5204799999999999 0.0061192810035166755
10 500 0.77191 0.036185928204206665
--- 445.20620489120483 seconds ---

In [ ]:
import numpy
import CO2_tree as co2t
import CO2_forest as co2f
import pickle
from scipy import sparse

from sklearn.metrics import accuracy_score
from thundersvm import *
from sklearn.model_selection import KFold

tree_deth = [5]
sratios = [0.]
sratios2 = [1.0]
fratios = [0.1]

C = [100]
criteria = ["gaussian"]

best_v_acc = 0.
for d in tree_deth:
    for sratio in sratios:
        for sratio2 in sratios2:
            for fratio in fratios:
                for c in C:
                    for ns in [0.]:
                        for cr in criteria:
                            v2_sc = []
                            t_scf = []
                            print ("Test carbon forest with tree deth= ", d+1, " C= ", c, " s ratio ", sratio," s ration 2" ,sratio2," f ratio ",fratio, "criterion=",cr)
                            for rn in range(1):
                                kf = KFold(n_splits=5, shuffle=True)
                                itr = 0
                                for train, test in kf.split(X_train_):
                                    print ("Step", rn)
                                    X_train = X_train_[train]
                                    X_test = X_train_[test]
                                    y_train = y_train_[train]
                                    y_test = y_train_[test]                                      
                                    itr += 1
                                    trc = co2f.CO2_forest(C=c, dual=False,tol = 0.0001,max_iter=1000,kernel=cr,max_deth=d,n_jobs=2,sample_ratio=1.0, feature_ratio = fratio,n_estimators=100,gamma=10.,dropout_low=sratio,dropout_high=sratio2,noise=ns,cov_dr=0.,criteria='gain')
                                    trc.sequential_fit(sparse.csr_matrix(X_train), y_train)
                                    Y_v_f = trc.sequential_predict(sparse.csr_matrix(X_test),use_weight = False)
                                    Y_t_f =  trc.sequential_predict(sparse.csr_matrix(X_train),use_weight = False)
                                    del trc
                                    #print(Y_v)

                                    try:
                                        t_scf.append(accuracy_score(y_train,Y_t_f))
                                        v2_sc.append(accuracy_score(y_test,Y_v_f))

                                    except Exception as e:
                                        print("Error:", e)
                                    break    
                            t_scf = numpy.asarray(t_scf)
                            v2_sc = numpy.asarray(v2_sc)

                            vres = v2_sc.mean()

                            #if vres > best_v_acc:
                            #    best_v_acc = vres
                            #    best_params[ds] = [d,fratio,c,cr,sratio,best_v_acc]     

                            print (cr,t_scf.mean(),vres)    
                                
                                
#print ("Best results:", best_params)   

Test carbon forest with tree deth=  6  C=  100  s ratio  0.0  s ration 2 1.0  f ratio  0.1 criterion= gaussian
Step 0
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready
Tree ready


In [ ]:
X_train.shape

In [4]:
cr,t_scf.mean(),vres

('gaussian', 0.84525, 0.5227)

In [ ]:
5 (array([0.5456]), array([0.888225])) C=2000
gaussian 0.98135 0.5538 C=2000 (30,0.8)
0.97465 0.5561 C=2000 (30,0.5)
0.94785 0.5534 (C=2000 (30,0.3))
gaussian 0.965325 0.5537 4 ( C=2000 30,0.8)
gaussian 0.96195 0.5579 3 (C=2000 30, 0.8)
3 ('gaussian', 0.97965, 0.5625) (C=2000 100, 0.8)
3 ('gaussian', 0.972325, 0.5725) (C=2000 100, 0.5)
3 ('gaussian', 0.9715, 0.5684) (С=2000 100, 0.5)
3 ('gaussian', 0.971175, 0.5707) (C=500, 100, 0.5) g = 0.1
3 ('gaussian', 0.713275, 0.5208) (C=2000 100, 0.1) g=0.1
3 ('gaussian', 0.7119, 0.525) (C=2000 100, 0.1) g=10
3 ('gaussian', 0.88095, 0.5578) (C=2000 100, 0.2) g=10
3 ('gaussian', 0.94025, 0.5595) (C=2000 100, 0.3) g=10
2 ('gaussian', 0.9905, 0.5634) (C=2000 100, 0.8) g=10
2 ('gaussian', 0.981625, 0.5735) (C=2000 100, 0.5) g=10
2 ('gaussian', 0.9719, 0.5628) (C=2000 100, 0.4) g=10
2 ('lin / gaussian', 0.89205, 0.5359) (C=2000 100, 0.5) g=10
4 ('gaussian', 0.587275, 0.493) (C=2000 100, 0.05) g=10
4 ('gaussian', 0.743, 0.5273) (C=2000 100, 0.1) g=10
4 ('gaussian', 0.4916, 0.4476) (C=100 100, 0.1) g=10
4 ('gaussian', 0.698825, 0.5295) (C=100 100, 0.5) g=10
5 ('gaussian', 0.722475, 0.5269) (C=100 100, 0.5) g=10
5 ('gaussian', 0.636025, 0.503) (C=100 100, 0.3) g=10

In [ ]:
([0.51078125], [0.8433984375]) C = 500

In [ ]:
!pip install sympy

In [ ]:
!pip install memory_profiler